In [1]:
# Importamos JSON
import json

# Guardamos las credenciales para consumir api de twitter (sacadas de apps.twiiter.com)
credentials = {
    "KEY": "c0Jlf7B9Ez56i6rsnBSeiBCii",
    "KEY_SECRET": "yuCAV2TdGrvitWBGQondppKaxNlQfnrc5krRCYvNpu3miY2gQQ",
    "TOKEN": "1279067386520440833-YLZvyZzPDZlQ3bGa5SH2fhugVPl4Cs",
    "ACCESS_SECRET": "DZuy6rzCJFZ4723jVl2gITgf2jCXr1cqlFRtQXFytrIKD"
}

credentials_PRO = {
    "KEY": "XqNe5tbaycDxrrIOVqHR1NoPk",
    "KEY_SECRET": "wahClTvDgP1Kb7l8QgHJD6DTFhhgY9usDfFaXSY5bnLdlrnv4q",
    "TOKEN": "1279025620287205377-OlltfTVwLqHRXklJPGAlAWvF7gC0bQ",
    "ACCESS_SECRET": "rJX1qRcToTyZq4Aqam18YjT1rbmFRhKvNflgB5RELz9aM"
}

credentials
credentials_PRO

{'KEY': 'XqNe5tbaycDxrrIOVqHR1NoPk',
 'KEY_SECRET': 'wahClTvDgP1Kb7l8QgHJD6DTFhhgY9usDfFaXSY5bnLdlrnv4q',
 'TOKEN': '1279025620287205377-OlltfTVwLqHRXklJPGAlAWvF7gC0bQ',
 'ACCESS_SECRET': 'rJX1qRcToTyZq4Aqam18YjT1rbmFRhKvNflgB5RELz9aM'}

In [2]:
# Intslamos Twythom
!pip install twython

In [6]:
# Instalamos geopy
!pip install geopy

In [5]:
#importamos las librerias necesarias
import pandas as pd

### consultar api tiempo real
from twython import TwythonStreamer

### Se guardara en un archivo .csv
import csv

## imortar libreria para buscar coordenadas
from geopy.geocoders import Nominatim



In [7]:
geolocator = Nominatim(user_agent="BD-Project-JJ")

coordinates =  []

location = geolocator.geocode("Port Moody, British Columbia")

if location:
    coordinates.append(location.latitude)
    coordinates.append(location.longitude)
else:
    no_location += 1
    coordinates.append(0)
    coordinates.append(0)
coordinates
        

[49.2822243, -122.8293424]

In [8]:
geolocator = Nominatim(user_agent="BD-Project-JJ")


#procesar los datos con filtro
def process_tweet(tweet):
    # titulos
    #date,hashtags,text,user,verifieduser,registeredon,followersuser,followinguser,tweetsuser,favoritestweetsuser,retweet,quote,reply,likes,language,device,location,latitude,longitude

        d = {}
        d['date'] = tweet['created_at']
        d["hashtags"] = [hashtag["text"] for hashtag in tweet['entities']['hashtags']]
        d['text'] = tweet['text']
        #datos user
        d['user'] = tweet['user']['screen_name']
        d['verified'] = tweet['user']['verified']
        d['created'] = tweet['user']['created_at']
        d['followers'] = tweet['user']['followers_count']
        d['following'] = tweet['user']['friends_count']
        d['tweets'] = tweet['user']['statuses_count']
        d['Favorites'] = tweet['user']['favourites_count']

        d['retweet'] = tweet['retweet_count']
        d['quote'] = tweet['quote_count']
        d['reply'] = tweet['reply_count'] 
        d['like'] = tweet['favorite_count']

        d['language'] = tweet['lang']

        ## guardar dispositivo
        if "android" in tweet['source']:
            d['device'] = "ANDROID"
        elif "iphone" in tweet['source']:
            d['device'] = "IPHONE"
        elif "ipad" in tweet['source']:
            d['device'] = "IPAD"
        else:
            d['device'] = "WEB APP"


        d['user_location'] = tweet['user']['location']
        ## guardar coordenadas ubicacion
        if tweet['user']['location']:
            coordinates = search_coordinates(tweet['user']['location'])
            d['latitude'] = coordinates[0]
            d['longitude'] = coordinates[1]
        else:
            d['latitude'] = "None"
            d['longitude'] = "None"

        return d

def search_coordinates(location):
    #Buscar y guardar coordenadas
    coordinates = []

    location = geolocator.geocode(location)

    if location:
        coordinates.append(location.latitude)
        coordinates.append(location.longitude)
    else:
        coordinates.append("None")
        coordinates.append("None")
    return coordinates

# Heredando clase
#se le pasa las credenciales (probar siempre con credentials normales.)
class Streamer(TwythonStreamer):

    #Datos recibidos
    def on_success(self, data):
        #si se necesita poner un condicional
        #if data['atributo'] == 'condicion':
        #print(data)
        tweet_data = process_tweet(data)
        self.save_tweet(tweet_data)

    #Error
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    #Guardar en un archivo el resultados
    def save_tweet(self, tweet):
        #reempalzar si no esta trabajando en visualstudio o no esta en la carpeta origen del proyecto
        #with open(r'tweets.csv', 'a', encoding="utf-8") as file: 
        with open(r'work/slides/data/tweets.csv', 'a', encoding="utf-8") as file:
            write = csv.writer(file)
            write.writerow(list(tweet.values()))

In [5]:
### Recoleccion de datos

In [14]:

# Intanciar
#stream =  Streamer(credentials['KEY'], credentials['KEY_SECRET'], credentials['TOKEN'], credentials['ACCESS_SECRET'])
stream =  Streamer(credentials_PRO['KEY'], credentials_PRO['KEY_SECRET'], credentials_PRO['TOKEN'], credentials_PRO['ACCESS_SECRET'])

#Iniciar consulta en tiempo real
stream.statuses.filter(track="covid")
# Para poder para la consulta y luego ver los resultados debe parar el proceso manualmente, es decir interrumpirlo.
# dando click en el boton de interrupt IPython kernel

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Indianapolis%2C+IN&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [15]:
tweets = pd.read_csv("work/slides/data/tweets.csv")
tweets

,date,hashtags,text,user,verifieduser,registeredon,followersuser,followinguser,tweetsuser,favoritestweetsuser,retweet,quote,reply,likes,language,device,location,latitude,longitude
0,Wed Jul 08 19:43:21 +0000 2020,[],RT @_lilhollywood: Lord please protect me and ...,haydeee_m,False,Thu Jun 21 02:02:11 +0000 2018,522,441,34043,16572,0,0,0,0,en,IPHONE,NaN,None,None
1,Wed Jul 08 19:43:21 +0000 2020,[],RT @itsJeffTiedrich: am I understanding this c...,Z_Castro_,False,Mon Sep 19 21:50:36 +0000 2011,412,1224,34883,36734,0,0,0,0,en,IPHONE,"Phoenix, AZ",33.4484367,-112.0741417
2,Wed Jul 08 19:43:21 +0000 2020,[],"RT @KyleKulinski: Japan has less than 1,000 co...",Lucid_Dreamer01,False,Mon Feb 28 23:38:18 +0000 2011,487,1729,10301,41028,0,0,0,0,en,IPHONE,NaN,None,None
3,Wed Jul 08 19:43:21 +0000 2020,[],RT @ApibOficial: Domingos Mahoro era president...,cianzeira,False,Wed May 09 20:12:39 +0000 2018,1411,464,6607,11253,0,0,0,0,pt,WEB APP,NaN,None,None
4,Wed Jul 08 19:43:21 +0000 2020,[],RT @Italianmike: So @HillaryClinton said she c...,JoyaMia00,False,Fri Jul 29 15:43:03 +0000 2016,22955,19801,83405,39015,0,0,0,0,en,IPAD,"Florida, USA",27.7567667,-81.4639835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,Wed Jul 08 22:08:02 +0000 2020,['Arizona'],RT @Cleavon_MD: I’m an ER Doctor. I’m not send...,MicheleMThomas,False,Thu Feb 26 01:36:49 +0000 2009,32,743,414,985,0,0,0,0,en,IPHONE,"Louisville, KY",38.2542376,-85.759407
739,Wed Jul 08 22:09:07 +0000 2020,['CubaPorLaVida'],RT @DiazCanelB: No one has been abandoned to t...,Alenamf19,False,Tue Sep 17 16:24:19 +0000 2019,6713,6233,53846,57689,0,0,0,0,en,ANDROID,"La Habana, Cuba",23.135305,-82.3589631
740,Wed Jul 08 22:09:07 +0000 2020,[],"RT @KyleKulinski: Japan has less than 1,000 co...",keithxo_,False,Thu Sep 03 19:18:45 +0000 2009,6837,3005,101044,39727,0,0,0,0,en,IPHONE,"Dallas, TX",32.7762719,-96.7968559
741,Wed Jul 08 22:09:07 +0000 2020,[],RT @itsJeffTiedrich: am I understanding this c...,jocilynvargas,False,Wed Sep 28 19:03:29 +0000 2016,116,151,8833,25031,0,0,0,0,en,IPHONE,NaN,None,None


In [16]:
import numpy as np

rows_no_coordinates = np.where(tweets['latitude']=="None")[0]

tweets_with_coordinates = tweets.drop(rows_no_coordinates)

tweets_with_coordinates

,date,hashtags,text,user,verifieduser,registeredon,followersuser,followinguser,tweetsuser,favoritestweetsuser,retweet,quote,reply,likes,language,device,location,latitude,longitude
1,Wed Jul 08 19:43:21 +0000 2020,[],RT @itsJeffTiedrich: am I understanding this c...,Z_Castro_,False,Mon Sep 19 21:50:36 +0000 2011,412,1224,34883,36734,0,0,0,0,en,IPHONE,"Phoenix, AZ",33.4484367,-112.0741417
4,Wed Jul 08 19:43:21 +0000 2020,[],RT @Italianmike: So @HillaryClinton said she c...,JoyaMia00,False,Fri Jul 29 15:43:03 +0000 2016,22955,19801,83405,39015,0,0,0,0,en,IPAD,"Florida, USA",27.7567667,-81.4639835
6,Wed Jul 08 19:43:21 +0000 2020,[],RT @msaludpy: Plasma convaleciente para tratam...,Nadia_Servian,False,Thu Apr 11 23:52:28 +0000 2013,1372,644,8319,1958,0,0,0,0,es,IPHONE,"Asuncion, Paraguay",-25.2800459,-57.6343814
8,Wed Jul 08 19:51:43 +0000 2020,[],pegar covid e morrer p mim seria tudo desculpa...,_clarineax,False,Sat Dec 30 10:22:41 +0000 2017,221,243,9634,18376,0,0,0,0,pt,WEB APP,"Recife, Brasil",-8.0584933,-34.8848193
9,Wed Jul 08 19:51:43 +0000 2020,[],It was never safe for your child to be in scho...,TheRoseBushes,False,Thu Jun 13 02:13:18 +0000 2013,836,3763,9879,4653,0,0,0,0,en,WEB APP,NE OH/ NW PA,41.1885838,-81.0170187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,Wed Jul 08 22:08:02 +0000 2020,[],Top story: How accounting firms can reactivate...,Stikks_Solomon,False,Fri Mar 01 18:23:45 +0000 2013,287,65,8905,7,0,0,0,0,en,WEB APP,Baldwin NY,40.6626725,-73.61030123947955
736,Wed Jul 08 22:08:02 +0000 2020,[],RT @CochraneCBC: The cost of covid in fiscal s...,BCAgroup,False,Thu Oct 08 08:29:43 +0000 2009,3321,3916,73459,14650,0,0,0,0,en,ANDROID,Ajax. Ontario Canada,43.850374,-79.024658
738,Wed Jul 08 22:08:02 +0000 2020,['Arizona'],RT @Cleavon_MD: I’m an ER Doctor. I’m not send...,MicheleMThomas,False,Thu Feb 26 01:36:49 +0000 2009,32,743,414,985,0,0,0,0,en,IPHONE,"Louisville, KY",38.2542376,-85.759407
739,Wed Jul 08 22:09:07 +0000 2020,['CubaPorLaVida'],RT @DiazCanelB: No one has been abandoned to t...,Alenamf19,False,Tue Sep 17 16:24:19 +0000 2019,6713,6233,53846,57689,0,0,0,0,en,ANDROID,"La Habana, Cuba",23.135305,-82.3589631


In [17]:
tweets_with_coordinates.to_csv("work/slides/data/tweetCoordinates.csv", sep='\t', encoding='utf-8')

In [18]:
# Intslamos gmplot
!pip install gmplot

In [19]:
latitude = pd.to_numeric(tweets_with_coordinates["latitude"], downcast="float")
longitude = pd.to_numeric(tweets_with_coordinates["longitude"], downcast="float")

latitude

1      33.448437
4      27.756767
6     -25.280046
8      -8.058494
9      41.188583
         ...    
735    40.662674
736    43.850372
738    38.254238
739    23.135305
740    32.776272
Name: latitude, Length: 446, dtype: float32

In [20]:
import gmplot

# instancia
gmap = gmplot.GoogleMapPlotter(30,0,3)

#insertar puntos en el mapa pasando una lista de coordenadas
gmap.heatmap(latitude.tolist(), longitude.to_list(), radius=20)


#Guardar el mapa
gmap.draw('work/slides/heatmap.html')